# day 5

https://adventofcode.com/5/day/5

In [2]:
import logging
import logging.config
import os

import yaml

In [3]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [4]:
FNAME = os.path.join('data', 'day05.txt')

LOGGER = logging.getLogger('day05')

## part 1

### problem statement:

#### loading data

In [5]:
test_data = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [6]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [14]:
class Map:
    def __init__(self, instructions: list[list[int]]):
        self.instructions = sorted(instructions, key=lambda x: x[1])

    def map(self, i: int) -> int:
        for (dst_start, src_start, l) in self.instructions:
            x = i - src_start
            if 0 <= x < l:
                return dst_start + x
        return i

    def range_map(self, seed_range: list[int]) -> list[list[int]]:
        # given the instructions, turn a range (i0, l) into possibly many new ranges after
        # applying all instructions: [(j0, l0), (j1, l1), ...]
        output_ranges = []
        i_inst = 0
        while True:
            try:
                instruction = self.instructions[i_inst]
            except IndexError:
                break

            remapped_range, seed_range, instruction_consumed = self.apply_instruction_to_range(instruction, seed_range)
            if remapped_range is not None:
                output_ranges.append(remapped_range)

            if seed_range is None:
                return output_ranges

            if instruction_consumed:
                i_inst += 1

        # at this point, if seed_range is not none, it means it wasn't touched by any instruction
        # and should be considered a straight pass-through
        output_ranges.append(seed_range)

        return output_ranges

    def range_list_map(self, seed_range_list: list[list[int]]) -> list[list[int]]:
        output_seed_range_list = []
        for seed_range in seed_range_list:
            output_seed_range_list += self.range_map(seed_range=seed_range)
        return output_seed_range_list

    def apply_instruction_to_range(self, instruction: list[int], seed_range: list[int]) -> tuple[list[int] | None, list[int] | None, bool]:
        """for a given input seed range and instruction, there are four options:

            1. no overlap at all
            2. complete overlap
            3. partial overlap: the first element is in the overlap
            4. partial overlap: the first element is not in the overlap

        turn each of those scenarios into a remapped range and remaining seed range

        returns:
            remapped_range
            seed_range_remaining
            instruction_consumed

        """
        i0, l_i = seed_range
        i1 = i0 + l_i

        dst0, src0, l_src = instruction
        src1 = src0 + l_src

        # no overlap at all
        if src1 < i0:
            remapped_range = None
            seed_range_remaining = seed_range
            instruction_consumed = True
        # first element is in the overlap
        elif src0 <= i0 < src1:
            j0 = dst0 + (i0 - src0)
            # complete overlap
            if i1 < src1:
                remapped_range = [j0, l_i]
                seed_range_remaining = None
                # only consumed when we've "passed" the right edge
                instruction_consumed = False
            # partial overlap: the first element is in the overlap
            else:
                # the overlapping area ends at src1 by definition
                l_overlap = src1 - i0
                l_remaining = i1 - src1
                remapped_range = [j0, l_overlap]
                seed_range_remaining = [src1, l_remaining]
                instruction_consumed = True
        # first element is not in the overlap
        else:
            # *some* element is in the overlap:
            if src0 < i1:
                remapped_range = [i0, src0 - i0]
                seed_range_remaining = [src0, i1 - src0]
                instruction_consumed = False
            # no overlap at all
            else:
                remapped_range = [i0, l_i]
                seed_range_remaining = None
                instruction_consumed = False

        return remapped_range, seed_range_remaining, instruction_consumed


class MapChain:
    def __init__(self, *maps):
        self.maps = maps

    def map(self, i: int) -> int:
        for map in self.maps:
            i = map.map(i)
        return i

    def range_list_map(self, seed_range_list: list[list[int]]) -> list[list[int]]:
        for map in self.maps:
            seed_range_list = map.range_list_map(seed_range_list)
        return seed_range_list


def str_to_inst(s: str) -> list[list[int]]:
    return [[int(_) for _ in line.split(' ')] for line in s.split('\n')[1:]]


def parse_data(d):
    (seeds,
     seed_to_soil,
     soil_to_fert,
     fert_to_water,
     water_to_light,
     light_to_temp,
     temp_to_hum,
     hum_to_loc) = d.split('\n\n')
    _, seeds = seeds.split(': ')
    seeds = [int(_) for _ in seeds.split(' ')]

    seed_to_soil = Map(instructions=str_to_inst(s=seed_to_soil))
    soil_to_fert = Map(instructions=str_to_inst(s=soil_to_fert))
    fert_to_water = Map(instructions=str_to_inst(s=fert_to_water))
    water_to_light = Map(instructions=str_to_inst(s=water_to_light))
    light_to_temp = Map(instructions=str_to_inst(s=light_to_temp))
    temp_to_hum = Map(instructions=str_to_inst(s=temp_to_hum))
    hum_to_loc = Map(instructions=str_to_inst(s=hum_to_loc))
    return seeds, (seed_to_soil,
                   soil_to_fert,
                   fert_to_water,
                   water_to_light,
                   light_to_temp,
                   temp_to_hum,
                   hum_to_loc)

In [15]:
(seeds,
 (seed_to_soil,
  soil_to_fert,
  fert_to_water,
  water_to_light,
  light_to_temp,
  temp_to_hum,
  hum_to_loc)) = parse_data(test_data)

assert seed_to_soil.map(0) == 0
assert seed_to_soil.map(1) == 1
assert seed_to_soil.map(48) == 48
assert seed_to_soil.map(49) == 49
assert seed_to_soil.map(50) == 52
assert seed_to_soil.map(51) == 53
assert seed_to_soil.map(96) == 98
assert seed_to_soil.map(97) == 99
assert seed_to_soil.map(98) == 50
assert seed_to_soil.map(99) == 51

assert seed_to_soil.map(79) == 81
assert seed_to_soil.map(14) == 14
assert seed_to_soil.map(55) == 57
assert seed_to_soil.map(13) == 13

In [16]:
assert seed_to_soil.range_map([79, 14]) == [[81, 14]]
assert seed_to_soil.range_map([55, 13]) == [[57, 13]]

In [18]:
all_maps = MapChain(seed_to_soil,
                    soil_to_fert,
                    fert_to_water,
                    water_to_light,
                    light_to_temp,
                    temp_to_hum,
                    hum_to_loc)

assert all_maps.map(79) == 82
assert all_maps.map(14) == 43
assert all_maps.map(55) == 86
assert all_maps.map(13) == 35

#### function def

In [20]:
def q_1(data):
    (seeds,
     (seed_to_soil,
      soil_to_fert,
      fert_to_water,
      water_to_light,
      light_to_temp,
      temp_to_hum,
      hum_to_loc)) = parse_data(d=data)
    all_maps = MapChain(seed_to_soil,
                        soil_to_fert,
                        fert_to_water,
                        water_to_light,
                        light_to_temp,
                        temp_to_hum,
                        hum_to_loc)
    seeds_to_loc = [[seed_idx, all_maps.map(seed_idx)] for seed_idx in seeds]
    return sorted(seeds_to_loc, key=lambda x: x[-1])[0][1]

#### tests

In [21]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 35
    LOGGER.setLevel(logging.INFO)

In [22]:
test_q_1()

#### answer

In [23]:
q_1(load_data())

551761867

## part 2

### problem statement:

#### function def

In [24]:
import itertools

def grouper(iterable, n, *, incomplete='fill', fillvalue=None):
    "Collect data into non-overlapping fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, fillvalue='x') --> ABC DEF Gxx
    # grouper('ABCDEFG', 3, incomplete='strict') --> ABC DEF ValueError
    # grouper('ABCDEFG', 3, incomplete='ignore') --> ABC DEF
    args = [iter(iterable)] * n
    if incomplete == 'fill':
        return itertools.zip_longest(*args, fillvalue=fillvalue)
    if incomplete == 'strict':
        return zip(*args, strict=True)
    if incomplete == 'ignore':
        return zip(*args)
    else:
        raise ValueError('Expected fill, strict, or ignore')

def generate_seed_numbers(seeds):
    for (seed_idx, l) in grouper(seeds, 2):
        LOGGER.warning(f"seed_idx: {seed_idx}")
        for i in range(l):
            yield seed_idx + i

seeds = [79, 14, 55, 13,]
assert list(generate_seed_numbers(seeds)) == list(range(79, 93)) + list(range(55, 68))

2023-12-05 21:12:07,919 WARNING  [day05.generate_seed_numbers:20] seed_idx: 79
2023-12-05 21:12:07,920 WARNING  [day05.generate_seed_numbers:20] seed_idx: 55


In [29]:
def q_2(data):
    (seeds,
     (seed_to_soil,
      soil_to_fert,
      fert_to_water,
      water_to_light,
      light_to_temp,
      temp_to_hum,
      hum_to_loc)) = parse_data(d=data)
    all_maps = MapChain(seed_to_soil,
                        soil_to_fert,
                        fert_to_water,
                        water_to_light,
                        light_to_temp,
                        temp_to_hum,
                        hum_to_loc)
    seed_ranges = list(grouper(seeds, 2))
    output_seed_ranges = all_maps.range_list_map(seed_ranges)
    range_start_vals = [i for (i, _) in output_seed_ranges]
    return min(range_start_vals)

#### tests

In [30]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 46
    LOGGER.setLevel(logging.INFO)

In [31]:
test_q_2()

#### answer

In [32]:
q_2(load_data())

57451709

fin